## **Instalar paquetes**

In [1]:
!pip install dash
!pip install dash-bootstrap-components
!pip install plotly
!pip install matplotlib
!pip install pandas
!pip install gurobipy
!pip install flask
!pip install pyngrok

## **Importar librerías**

In [2]:
# ----------------------------------------
# Librerías estándar de Python
# ----------------------------------------
import os
import io
import time
import json
import base64
import urllib.request
import threading
from collections import Counter, defaultdict
import math

# ----------------------------------------
# Dash y componentes relacionados
# ----------------------------------------
import dash
import dash.html as html
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html, Input, Output, State, ctx, no_update
from dash.exceptions import PreventUpdate

# ----------------------------------------
# Visualización de datos
# ----------------------------------------
import plotly.express as px
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import rcParams
import matplotlib.font_manager as fm
from matplotlib.colors import to_hex
from matplotlib import cm


# ----------------------------------------
# Manipulación y optimización de datos
# ----------------------------------------
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# ----------------------------------------
# Integración con servidor y túneles
# ----------------------------------------
from flask import Flask
from pyngrok import ngrok, conf


## **Auto-token ngrok**

In [3]:
# Iniciar túnel ngrok con tu AUTHTOKEN
ngrok.set_auth_token("TU AUTHTOKEN AQUI")

## **Modelo**

In [4]:
def modelo(data, time_limit, equit=True):
    options = {
        "WLSACCESSID": "TU WLSACCESSID AQUI",
        "WLSSECRET": "TU WLSSECRET AQUI",
        "LICENSEID": licenseid
    }

    with gp.Env(params=options) as env, gp.Model(env=env) as m:

        # ----------------------------------------
        # Sets
        # ----------------------------------------
        E = data['Employees']
        D = data['Desks']
        J = data['Days']
        G = data['Groups']
        Z = data['Zones']
        D_z = data['Desks_Z']
        D_e = data['Desks_E']
        E_g = data['Employees_G']
        J_e = data['Days_E']
        E_d = defaultdict(list)
        for e, desks in D_e.items():
            for d in desks:
                E_d[d].append(e)
        E_d = dict(E_d)

        days = ['L', 'Ma', 'Mi', 'J', 'V']
        NJ_e = defaultdict(list)
        for e in E:
            NJ_e[e] = list(set(days)-set(J_e[e]))
        NJ_e = dict(NJ_e)

        # ----------------------------------------
        # Decision variables
        # ----------------------------------------
        x = m.addVars([(e, d, j) for e in E for d in D_e[e] for j in J], vtype=gp.GRB.BINARY, name="x")
        u = m.addVars([(g, j) for g in G for j in J], vtype=gp.GRB.BINARY, name="u")
        y = m.addVars([(g, z, j) for g in G for z in Z for j in J], vtype=gp.GRB.BINARY, name="y")
        w = m.addVars([(g, j) for g in G for j in J], vtype=gp.GRB.BINARY, name="w")
        c = m.addVars([(e) for e in E], name="c")
        mc = m.addVars([(e, d) for e in E for d in D], vtype=gp.GRB.BINARY, name="mc")

        # ----------------------------------------
        # Constraints
        # ----------------------------------------
        # O_: Para FO
        m.addConstrs((c[e] - 10*(1 - mc[e, d]) <= gp.quicksum(x[e, d, j] for j in J) for e in E for d in D_e[e]), name="V_Consistency")
        m.addConstrs((gp.quicksum(mc[e, d] for d in D_e[e]) <= 1 for e in E), name="V_Consistency_2")
        m.addConstrs((gp.quicksum(mc[e, d] for d in D_e[e]) >= 1 for e in E), name="V_Consistency_3")
        # O_1: Que todo el grupo vaya (al menos) un día
        m.addConstrs((gp.quicksum(x[e, d, j] for e in E_d[d]) <= 1 for j in J for d in D), name="C0_1EperD")
        m.addConstrs((u[g, j] <= gp.quicksum(x[e, d, j] for e in E_g[g] for d in D_e[e])/len(E_g[g]) for g in G for j in J), name="C1_everyonemustgo")
        m.addConstrs((gp.quicksum(u[g, j] for j in J) >= 1 for g in G), name="C1_meetingday")
        # O_2: Compatibilidades puesto de trabajo x colaborador
        m.addConstrs((gp.quicksum(x[e, d, j] for d in D_e[e]) <= 1 for j in J for e in E), name="C2_compatibility")
        # O_3: Que no hayan miembros aislados
        m.addConstrs((x[e, d, j] <= y[g, z, j] for g in G for e in E_g[g] for j in J for d in D_e[e] for z in Z if d in D_z[z]), name="C3_isolation_1")
        m.addConstrs(((gp.quicksum(y[g, z, j] for z in Z)-1)/len(Z) <= w[g, j] for j in J for g in G), name="C3_isolation_2")
        m.addConstrs((2*w[g, j] <= gp.quicksum(x[e, d, j] for e in E_g[g] for d in D_e[e] if d in D_z[z]) + 10 * (1 - y[g, z, j]) for j in J for g in G for z in Z), name="C3_isolation_3")

        # ----------------------------------------
        # Time allocation
        # ----------------------------------------
        t_FO1 = 1/6*time_limit
        t_FO2 = 1/6*time_limit
        t_FO3 = 4/6*time_limit

        t_i_FO1 = time_limit
        t_i_FO2 = time_limit
        t_i_FO3 = time_limit

        if len(E) < 30:
            pass
        if len(E) < 50:
            t_i_FO3 = 1/3*t_FO3
        elif len(E) < 90:
            t_i_FO2 = 1/3*t_FO2
            t_i_FO3 = 1/3*t_FO3
        else:
            t_i_FO1 = 1/3*t_FO1
            t_i_FO2 = 1/3*t_FO2
            t_i_FO3 = 0*t_FO3

        reltol_FO1 = 0
        if equit:
            reltol_FO1 = 0.05

        # ----------------------------------------
        # Solve
        # ----------------------------------------

        start = time.time()

        # Primer objetivo: maximizar equidad o cobertura
        if equit:
            obj1 = gp.quicksum((gp.quicksum(x[e, d, j] for d in D_e[e] for j in J_e[e]) - gp.quicksum(x[e, d, k] for d in D_e[e] for k in NJ_e[e]))/len(J_e[e]) for e in E)
        else:
            obj1 = gp.quicksum(gp.quicksum(x[e, d, j] for d in D_e[e] for j in J_e[e]) - gp.quicksum(x[e, d, k] for d in D_e[e] for k in NJ_e[e]) for e in E)
        m.setObjective(obj1, GRB.MAXIMIZE)
        m.setParam("ImproveStartTime", t_i_FO1)
        m.setParam("TimeLimit", t_FO1)
        m.optimize()
        obj1_val = m.ObjVal

        # Segundo objetivo: minimizar zonas usadas
        obj2 = gp.quicksum(y[g, z, j] for g in G for z in Z for j in J)
        m.setObjective(obj2, GRB.MINIMIZE)
        m.addConstr(obj1 >= (1-reltol_FO1) * obj1_val, "Obj1_Tolerance")
        m.setParam("ImproveStartTime", t_i_FO2)
        m.setParam("TimeLimit", t_FO2)
        m.optimize()
        obj2_val = m.ObjVal

        # Tercer objetivo: maximizar consistencia
        duration = time.time() - start
        t_FO3 = time_limit - duration
        t_i_FO3 = 1/3*t_FO3

        obj3 = gp.quicksum(c[e] for e in E)
        m.setObjective(obj3, GRB.MAXIMIZE)
        m.addConstr(obj1 >= (1-reltol_FO1) * obj1_val, "Obj1_Tolerance")
        m.addConstr(obj2 <= 1 * obj2_val, "Obj2_Tolerance")
        m.setParam("ImproveStartTime", t_i_FO3)
        m.setParam("TimeLimit", t_FO3)
        m.optimize()
        obj3_val = m.ObjVal

        # ----------------------------------------
        # Process results
        # ----------------------------------------
        assignements = defaultdict(list)
        for j in J:
            for g in G:
                for e in E_g[g]:
                    for d in D_e[e]:
                        if x[e, d, j].X > 0.5:
                            for z in Z:
                                if d in D_z[z]:
                                    assignements[j].append((z, d, e, g))
                                    break
        meeting = defaultdict(list)
        for g in G:
            for j in J:
                if u[g, j].X > 0.5:
                    meeting[g].append(j)

        if equit:
            totalFO1 = len(E)
        else:
            totalFO1 = sum({day: len(assignments) for day, assignments in assignements.items()}.values())

        totalFO3 = sum({day: len(assignments) for day, assignments in assignements.items()}.values())
        return obj1.getValue(), obj2_val, obj3_val, meeting, assignements, totalFO1, totalFO3


## **Funciones**

In [5]:
def asignar_colores_zonas(zonas):
    num_zonas = len(zonas)
    cmap = cm.get_cmap('tab20')
    colores_base = cmap.colors  # Aquí están los 20 colores reales como lista de tuples

    # Asegura 20 colores (si hay más zonas, se repiten)
    colores = [to_hex(colores_base[i % len(colores_base)]) for i in range(num_zonas)]

    return {zona: colores[i] for i, zona in enumerate(sorted(zonas))}
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def construir_tabla_html(asignaciones, empleados_grupo, zona_colores, meeting):
    dias = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]
    dias_abreviados = ["L", "Ma", "Mi", "J", "V"]
    html_rows = []

    # Mapear días de reunión por grupo
    dias_meeting_set = {(grupo, dia): True for grupo, dias_grupo in meeting.items() for dia in dias_grupo}

    # Crear acceso rápido a asignaciones
    asignaciones_dict = {}
    for dia, lista_asign in asignaciones.items():
        for asignacion in lista_asign:
            empleado_id = asignacion[2]
            asignaciones_dict[(dia, empleado_id)] = (asignacion[0], asignacion[1])  # (zona, escritorio)

    for grupo, empleados in empleados_grupo.items():
        html_rows.append(f"<tr><td colspan='{len(dias)+1}' style='background:#eee;font-weight:bold;text-align:center'>Grupo {grupo}</td></tr>")
        html_rows.append("<tr><th></th>" + "".join(f"<th style='text-align:center'>{nombre}</th>" for nombre in dias) + "</tr>")

        # Fila de íconos de reunión
        icon_row = ["<tr><td></td>"]
        for dia_abrev in dias_abreviados:
            icon_row.append("<td style='text-align:center;'><i class='fas fa-users' style='color:#555;'></i></td>" if dias_meeting_set.get((grupo, dia_abrev)) else "<td></td>")
        icon_row.append("</tr>")
        html_rows.append("".join(icon_row))

        # Filas por empleado
        for emp in empleados:
            row_cells = [f"<tr><td style='text-align:center;font-weight:bold'>{emp}</td>"]
            for dia_abrev in dias_abreviados:
                key = (dia_abrev, emp)
                if key in asignaciones_dict:
                    zona, escritorio = asignaciones_dict[key]
                    color = zona_colores.get(zona, "white")
                    row_cells.append(f"<td style='background-color:{color};text-align:center;border:1px solid #ccc'>{escritorio}</td>")
                else:
                    row_cells.append("<td style='background-color:white;text-align:center;border:1px solid #ccc'></td>")
            row_cells.append("</tr>")
            html_rows.append("".join(row_cells))

        # Separación entre grupos
        html_rows.append(f"<tr><td colspan='{len(dias)+1}' style='height:10px;'></td></tr>")

    return (
        "<table style='border-collapse: collapse; width: 100%; font-size: 0.9rem; margin-bottom: 20px'>"
        + "".join(html_rows)
        + "</table>"
    )

# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def construir_leyenda_colores(zona_colores):
    bloques = "".join(
        f"<div style='display: flex; align-items: center; margin: 0 10px;'>"
        f"<div style='width:20px;height:20px;background:{color};border:1px solid #aaa;'></div>"
        f"<span style='margin-left:5px'>{zona}</span></div>"
        for zona, color in zona_colores.items()
    )

    return f"""
    <div style='width: 100%; text-align: center; padding: 12px 0; background-color: #eee; margin-bottom: 15px; box-shadow: 0 1px 3px rgba(0,0,0,0.05);'>
        <div style='font-weight: bold; margin-bottom: 8px;'>Leyenda de colores</div>
        <div style='display: flex; justify-content: center; flex-wrap: wrap;'>{bloques}</div>
    </div>"""

# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def construir_plot_html(asignaciones, empleados_grupo, zonas, meeting, desks_z):
    # Función auxiliar: convierte un string RGB a tupla normalizada [0, 1]
    def parse_rgb_string(rgb_str):
        nums = [int(x) for x in rgb_str.strip('rgb()').split(',')]
        return tuple([v / 255 for v in nums])

    # -----------------------------------------------
    # Descargar las fuentes Poppins si no existen
    # -----------------------------------------------
    poppins_regular_path = "/tmp/Poppins-Regular.ttf"
    poppins_bold_path = "/tmp/Poppins-Bold.ttf"

    if not os.path.exists(poppins_regular_path):
        urllib.request.urlretrieve(
            "https://github.com/google/fonts/raw/main/ofl/poppins/Poppins-Regular.ttf",
            poppins_regular_path
        )

    if not os.path.exists(poppins_bold_path):
        urllib.request.urlretrieve(
            "https://github.com/google/fonts/raw/main/ofl/poppins/Poppins-Bold.ttf",
            poppins_bold_path
        )

    # Registrar las fuentes descargadas en matplotlib
    fm.fontManager.addfont(poppins_regular_path)
    fm.fontManager.addfont(poppins_bold_path)
    fm._load_fontmanager(try_read_cache=False)  # Refrescar caché de fuentes

    # Configurar fuentes globales para matplotlib
    plt.rcParams['font.family'] = 'Poppins'
    plt.rcParams['font.sans-serif'] = 'Poppins'

    poppins_regular = fm.FontProperties(fname=poppins_regular_path)
    poppins_bold_title = fm.FontProperties(fname=poppins_bold_path, size=16)

    rcParams['font.family'] = poppins_regular.get_name()
    rcParams['axes.titlepad'] = 25

    # -----------------------------------------------
    # Configurar colores base y asignar a grupos
    # -----------------------------------------------
    num_grupos = len(empleados_grupo)

    cmap = cm.get_cmap('tab20')
    colores_base = cmap.colors

    base_colors_hex = [to_hex(c) for c in colores_base]

    # Asigna color a cada grupo, repitiendo si hay más de 20 grupos
    group_colors = {
        g: base_colors_hex[i % len(base_colors_hex)]
        for i, g in enumerate(empleados_grupo.keys())
    }

    # Etiquetas y claves de días de la semana
    day_labels = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
    day_keys = ['L', 'Ma', 'Mi', 'J', 'V']

    # -----------------------------------------------
    # Calcular parámetros para la leyenda de colores
    # -----------------------------------------------
    box_width = 0.02
    box_height = 0.25
    text_gap = 0.01
    spacing_x = 0.25
    spacing_y = 0.5
    max_grupos_por_fila = 4
    filas = math.ceil(len(group_colors)) // max_grupos_por_fila
    leyenda_altura = 0.01 * filas

    # -----------------------------------------------
    # Determinar dimensiones de la figura principal
    # -----------------------------------------------
    num_zonas = len(zonas)
    subplot_width = 4

    max_desks = 0
    for zone, desks in desks_z.items():
        if len(desks) > max_desks:
            max_desks = len(desks)
    rows_zone = math.ceil(max_desks / 3)
    subplot_height_per_zona = 3 * rows_zone
    height = subplot_height_per_zona * num_zonas
    width = 5 * subplot_width

    # Configurar proporción de la leyenda y los subplots
    ratio_leg_rows = (0.3 * filas) / (subplot_height_per_zona * num_zonas)

    # Ajustar separación vertical según número de filas en la leyenda
    if filas == 1:
        hhh = -0.25
    elif filas == 2:
        hhh = -0.55
    elif filas == 3:
        hhh = -0.55
    elif filas == 4:
        hhh = 0.03
    else:
        hhh = 0.04

    # Crear figura y grilla de subplots
    gs = plt.GridSpec(2, 5, height_ratios=[ratio_leg_rows, 1 - ratio_leg_rows], hspace=hhh)
    fig = plt.figure(figsize=(width, height), dpi=150)

    ax_legend = fig.add_subplot(gs[0, :])
    ax_legend.axis('off')

    axs = [fig.add_subplot(gs[1, i]) for i in range(5)]
    y_min = 0.9 - (filas + 1) * spacing_y
    ax_legend.set_ylim(y_min - 0.1, 1)

    # -----------------------------------------------
    # Dibujar bloques de la leyenda de colores
    # -----------------------------------------------
    for i, (grp, color) in enumerate(group_colors.items()):
        fila = i // max_grupos_por_fila
        col = i % max_grupos_por_fila
        x = 0.08 + col * spacing_x
        y = 0.8 - fila * spacing_y

        ax_legend.add_patch(
            patches.Rectangle(
                (x, y - box_height / 2), box_width, box_height,
                facecolor=color,
                edgecolor='black',
                linewidth=0.5
            )
        )
        ax_legend.text(
            x + box_width + text_gap, y,
            f"Grupo {grp}",
            ha='left', va='center', fontsize=14,
            fontproperties=poppins_regular,
            color='#333'
        )

    # -----------------------------------------------
    # Dibujar los subplots de cada día de la semana
    # -----------------------------------------------
    for idx, (label, key) in enumerate(zip(day_labels, day_keys)):
        ax = axs[idx]
        ax.set_title(label, fontproperties=poppins_bold_title, color='#222', pad=20, x=0.43 + 0.005 * idx)
        ax.axis('off')
        ax.set_xlim(-0.2, 3.5)
        ax.set_ylim(-len(zonas) * 4 + 5, 5.3)
        ax.set_facecolor('white')
        ax.set_aspect('equal')

        # Línea divisoria vertical entre días (opcional)
        if idx < len(day_labels) - 1:
            ax.axvline(x=3.4, ymin=0, ymax=1, color='#ccc',
                       linestyle='--', linewidth=1)

        # Agrupar asignaciones por zona para el día actual
        day_assignments = asignaciones.get(key, [])
        assign_by_zone = defaultdict(list)
        for z, d, e, g in day_assignments:
            assign_by_zone[z].append((d, e, g))

        # Dibujar escritorios por zona
        for i_z, z in enumerate(sorted(zonas)):
            zone_offset = i_z * 4
            assigned_desks = {d for d, _, _ in assign_by_zone.get(z, [])}
            zone_desks = desks_z.get(z, [])

            for i, desk in enumerate(sorted(zone_desks, key=lambda d: int(d[1:]))):
                xi = i % 3
                yi = 4 - (i // 3) - zone_offset

                if desk in assigned_desks:
                    # Escritorio asignado
                    emp = next((e for d_, e, _ in assign_by_zone[z] if d_ == desk), "")
                    grp = next((g for d_, _, g in assign_by_zone[z] if d_ == desk), "")

                    rect = patches.Rectangle(
                        (xi, yi), 0.9, 0.9,
                        linewidth=1,
                        edgecolor='#bbb',
                        facecolor=group_colors.get(grp, '#ddd'),
                        alpha=0.9
                    )
                    ax.add_patch(rect)
                    ax.text(xi + 0.45, yi + 0.68, desk, ha='center', va='center',
                            fontsize=8, color='#333', fontproperties=poppins_regular)
                    ax.text(xi + 0.45, yi + 0.4, emp, ha='center', va='center',
                            fontsize=11, fontweight='bold', color='#111', fontproperties=poppins_regular)
                else:
                    # Escritorio libre
                    rect = patches.Rectangle(
                        (xi, yi), 0.9, 0.9,
                        linewidth=1,
                        edgecolor='#ddd',
                        facecolor='#f9f9f9',
                        alpha=0.7,
                        linestyle=':'
                    )
                    ax.add_patch(rect)
                    ax.text(xi + 0.45, yi + 0.68, desk, ha='center', va='center',
                            fontsize=8, color='#bbb', fontproperties=poppins_regular)
                    ax.text(xi + 0.45, yi + 0.4, "LIBRE", ha='center', va='center',
                            fontsize=9, color='#999', style='italic', fontproperties=poppins_regular)

            # Etiqueta de zona y línea de separación
            ax.text(-0.2, 5 - zone_offset - 1.5, f"{z}", fontsize=10,
                    rotation=90, va='center', ha='center', color='#444',
                    fontproperties=poppins_regular)
            ax.hlines(5 - zone_offset - 3.5, xmin=-0.2, xmax=3.2,
                      colors='#eee', linestyles='dotted', linewidth=1)

    # -----------------------------------------------
    # Ajustar márgenes y exportar como imagen base64
    # -----------------------------------------------
    plt.subplots_adjust(top=0.92, bottom=0.05)

    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=300, bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode("utf-8")

    # Devolver componente HTML con la imagen generada
    return html.Img(
        src="data:image/png;base64," + img_base64,
        style={
            'width': '100%',
            'maxWidth': '1300px',
            'margin': '0 auto',
            'border': 'none',
            'padding': '10px',
            'backgroundColor': 'transparent'
        }
    )


# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def grafico_donut(valor, color="#28a745"):
    fig = go.Figure(data=[go.Pie(
        values=[valor, 100 - valor], hole=0.7, sort=False, rotation=90, textinfo='none',
        marker=dict(colors=[color, "lightgray"], line=dict(color='white', width=2))
    )])

    fig.update_layout(
        margin=dict(t=10, b=10, l=10, r=10), showlegend=False,
        height=120, width=120, paper_bgcolor='rgba(0,0,0,0)',
        annotations=[dict(text=f"<b>{valor:.2f}%</b>", x=0.5, y=0.5, showarrow=False,
                          font=dict(size=18, family="Poppins, sans-serif", color="#343a40"))]
    )

    return dcc.Graph(figure=fig, config={"displayModeBar": False}, style={"margin": "0 auto", "display": "block"})

# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def popover(texto, target):
    return dbc.Popover(texto, target=target, body=True, trigger="hover", placement="right", style={"fontFamily": "Poppins, sans-serif"})


## **Dash app**

In [6]:
# ----------------------------------------
# Configuración inicial
# ----------------------------------------
conf.get_default().region = "us"
ngrok.kill()

app = Dash(
    __name__,
    external_stylesheets=[
        dbc.themes.ZEPHYR,
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css",
        "https://fonts.googleapis.com/css2?family=Poppins&display=swap"
    ]
)
app.title = "FlexWork"

# ----------------------------------------
# Layout principal
# ----------------------------------------
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),

    html.Div([
        html.Div([
            html.Img(src="https://raw.githubusercontent.com/jruedaa/InduPeers/main/imagenes/LogoASOCIO.png", style={"height": "80px", "marginRight": "10px"}),
            html.Img(src="https://raw.githubusercontent.com/jruedaa/InduPeers/main/imagenes/LogoUdeA.png", style={"height": "80px"})
        ], style={
            "position": "absolute",
            "top": "50%",
            "right": "20px",
            "transform": "translateY(-50%)",
            "display": "flex",
            "gap": "10px"
        }),

        html.H1("FlexWork", className="text-center", style={"fontSize": "2.5rem", "fontWeight": "bold", "margin": "0"}),
        html.P("InduPeers", className="text-center", style={"fontSize": "1.3rem", "color": "#35944b", "fontWeight": "bold", "margin": "0"})
    ], style={"position": "relative", "paddingTop": "10px", "paddingBottom": "10px"}),

    # Barra de navegación
    dbc.Navbar(
        dbc.Container([
            dbc.NavbarBrand(""),
            dbc.Nav([
                dbc.NavLink("Inicio", href="/", id="link-inicio", active="exact", className="mx-3", style={"fontSize": "1.2rem"}),
                dbc.NavLink("Ejecución", href="/ejecucion", id="link-ejecucion", active="exact", className="mx-3", style={"fontSize": "1.2rem"}),
            ], pills=True, className="mx-auto justify-content-center")
        ]),
        color="#444", dark=True, className="mb-4 rounded",
        style={"maxWidth": "1500px", "margin": "0 auto"}
    ),

    # Contenedor dinámico de páginas
    html.Div(id='page-content', className='container')
], style={"fontFamily": "'Poppins', sans-serif"})

# ----------------------------------------
# Contenido página de Inicio
# ----------------------------------------
inicio_content = html.Div([

    dbc.Card([
        dbc.CardHeader(html.H5("Reto ASOCIO 2025", className="fw-bold text-center", style={"color": "#026937"}), style={"paddingTop": "20px", "paddingBottom": "20px"}),
        dbc.CardBody([
            html.P(
                "La Universidad busca una solución innovadora para optimizar la asignación de puestos de trabajo en un esquema híbrido. "
                "A raíz de la implementación del teletrabajo, los colaboradores comparten espacios según su calendario de presencialidad, lo que ha generado desafíos logísticos. "
                "El reto consiste en desarrollar una herramienta de apoyo a la toma de decisiones que permita asignar, de forma eficiente y equitativa, "
                "los días de asistencia, los espacios de trabajo y la distribución de los equipos, teniendo en cuenta sus preferencias, funciones y requisitos específicos.",
                style={"textAlign": "justify", "margin": "0 auto", "maxWidth": "2000px"}
            )
        ])
    ], className="mb-4 shadow-sm"),

    dbc.Alert(
        "Una solución inteligente y eficiente que optimiza la asignación de espacios y días de presencialidad, "
        "brindando flexibilidad y mejorando la experiencia del colaborador. Agiliza tus procesos con FlexWork y olvídate de procesos innecesarios.",
        style={"backgroundColor": "rgba(141, 198, 63, 0.15)", "border": "1px solid #8dc63f", "color": "#4d7a10", "maxWidth": "1000px", "textAlign": "justify", "margin": "25px auto 0 auto"},
        className="text-center mb-4"
    ),

    dbc.Card([
        dbc.CardHeader(html.H5("Autoras/Desarrolladoras", className="fw-bold text-center", style={"color": "#026937"}), style={"paddingTop": "20px", "paddingBottom": "20px"}),
        dbc.CardBody([
            html.Div([
                *[
                    html.Div([
                        dbc.Button(nombre, color="secondary", href=linkedin, target="_blank", style={"fontSize": "1.1rem"}),
                        html.P([html.I(className="fas fa-envelope", style={"color": "#70205b", "marginRight": "5px"}), correo],
                               className="text-center", style={"marginTop": "5px", "fontSize": "0.85rem"})
                    ], className="mx-3 text-center") for nombre, linkedin, correo in [
                        ("Daniela Alcázar Nieto", "http://www.linkedin.com/in/daniela-alcázar-nieto-", "daniela.alcazar@udea.edu.co"),
                        ("Esteffany Peña Puentes", "https://www.linkedin.com/in/esteffany-p-puentes", "esteffany.penap@udea.edu.co"),
                        ("Juliana Rueda Arango", "https://www.linkedin.com/in/juliana-rueda-arango-b372082b9", "juliana.rueda2@udea.edu.co")
                    ]
                ]
            ], className="d-flex justify-content-center mt-3 flex-wrap")
        ])
    ], className="mb-4 shadow-sm"),

    dbc.Card([
        dbc.CardHeader(html.H5("Información adicional", className="fw-bold text-center", style={"color": "#026937"}), style={"paddingTop": "20px", "paddingBottom": "20px"}),
        dbc.CardBody([
            html.Div([
                dbc.Button(
                    [html.I(className="fas fa-file-alt me-2"), "Informe"],
                    href="https://github.com/jruedaa/InduPeers/raw/9186f02142c5d4f2912f6227cbec6aff8d8b3152/InduPeers%20-%20Informe%20Reto%20ASOCIO.pdf",
                    target="_blank",
                    download="InduPeers_Informe.pdf",  # sugerencia de nombre de archivo
                    style={"backgroundColor": "#0e7774", "borderColor": "#0e7774", "color": "white"},
                    className="me-3"
                ),
                dbc.Button([html.I(className="fab fa-github me-2"), "Repositorio GitHub"],
                           color="dark", href="https://github.com/jruedaa/InduPeers", target="_blank")
            ], className="d-flex justify-content-center mt-2")
        ])
    ], className="mb-4 shadow-sm")
])

# ----------------------------------------
# Contenido página de Ejecución
# ----------------------------------------
ejecucion_content = dbc.Row([

    # Isla 1: Carga de datos
    dbc.Col(dbc.Card([
        dbc.CardHeader(html.H5("Carga de datos", className="fw-bold text-center", style={"color": "#026937"})),
        dbc.CardBody([
            html.Div([
                dbc.Label(["Tiempo máximo de ejecución", html.Span(html.I(className="fas fa-info-circle", style={"color": "#137598"}), id="info-tiempo", className="text-primary ms-2", style={"cursor": "pointer"})], className="fw-bold"),
                html.Div(dbc.InputGroup([
                    dbc.Input(id="tiempo-max", type="number", min=1, value=5, step=1, style={"width": "100px", "textAlign": "center", "border": "none", "flex": "1"}),
                    html.Span("minutos", style={"padding": "8px 12px", "backgroundColor": "white", "border": "none", "color": "#555", "fontWeight": "500", "display": "flex", "alignItems": "center", "justifyContent": "center", "height": "38px"})
                ], style={"border": "1px solid #35944b", "borderRadius": "6px", "overflow": "hidden", "display": "flex", "alignItems": "center", "width": "fit-content"}), className="d-flex justify-content-center mb-3"),
                popover("Define cuánto tiempo puede tomar la ejecución del algoritmo antes de detenerse automáticamente.", "info-tiempo")
            ]),
            html.Hr(className="my-2"),
            html.Div([
                dbc.Label(["Enfoque de asignación de días", html.Span(html.I(className="fas fa-info-circle", style={"color": "#137598"}), id="info-enfoque", className="text-primary ms-2", style={"cursor": "pointer"})], className="fw-bold"),
                html.Div(dbc.ButtonGroup([
                    dbc.Button("Equitativo", id="btn-alta", n_clicks=1, style={"backgroundColor": "#35944b", "borderColor": "#35944b", "color": "white", "fontWeight": "bold"}),
                    dbc.Button("Igual valor", id="btn-baja", n_clicks=0, style={"backgroundColor": "white", "borderColor": "#35944b", "color": "#35944b", "fontWeight": "bold"})
                ]), className="d-flex justify-content-center mb-3"),
                popover(
                    "Define cómo se priorizan las preferencias: El enfoque Equitativo asigna el mismo peso a cada persona, sin importar cuántos días solicite. El enfoque Igual Valor busca maximizar la cantidad total de preferencias satisfechas.",
                    "info-enfoque"
                )
            ]),
            html.Hr(className="my-2"),
            html.Div([
                dbc.Label(["Cargar archivo .json", html.Span(html.I(className="fas fa-info-circle", style={"color": "#137598"}), id="info-json", className="text-primary ms-2", style={"cursor": "pointer"})], className="fw-bold"),
                dcc.Upload(
                    id="upload-json",
                    children=html.Div([
                        html.I(className="fas fa-upload", style={"fontSize": "1.2rem", "marginBottom": "8px", "color": "#aaa"}),
                        html.Div("Arrastra o haz clic para cargar el archivo", style={"fontSize": "0.85rem", "color": "#aaa"})
                    ]),
                    style={"width": "100%", "height": "90px", "lineHeight": "20px", "borderWidth": "1px", "borderStyle": "dashed", "borderRadius": "5px", "textAlign": "center", "marginBottom": "10px", "paddingTop": "10px", "borderColor": "#ccc", "backgroundColor": "#f8f9fa", "color": "#aaa"},
                    accept=".json"
                ),
                popover("Carga un archivo de entrada en formato .json que contenga los datos requeridos por el sistema.", "info-json")
            ]),
            html.Div(id="json-output"),
            html.Div(dbc.Button("Ejecutar", id="btn-ejecutar", className="mt-2 w-100 fw-bold", style={"backgroundColor": "#069a7e", "borderColor": "#069a7e", "color": "white"}))
        ])
    ], className="shadow-sm p-3 rounded", style={"minHeight": "500px"}), width=3),


    # Isla 2 y 3: Indicadores y Resultados
    dbc.Col([
        dbc.Card([
            dbc.CardHeader(html.H5("Indicadores", className="fw-bold text-center", style={"color": "#026937"})),
            dbc.CardBody(html.Div([
                html.Div([
                    html.H6("Satisfacción días preferidos", className="text-center mb-3 fw-bold", style={"fontFamily": "Poppins, sans-serif"}),
                    html.Div(id="indicador-1", children="-", className="d-flex justify-content-center")
                ], className="pe-3", style={"flex": "1"}),
                html.Div(style={"width": "1px", "backgroundColor": "#ccc", "alignSelf": "stretch", "margin": "0 20px"}),
                html.Div([
                    html.H6("Consistencia de los escritorios", className="text-center mb-3 fw-bold", style={"fontFamily": "Poppins, sans-serif"}),
                    html.Div(id="indicador-3", children="-", className="d-flex justify-content-center")
                ], className="ps-3", style={"flex": "1"})
            ], className="d-flex align-items-start justify-content-center"))
        ], className="shadow-sm p-3 rounded mb-3"),

        dbc.Card([
            dbc.CardHeader(html.H5(["Resultados", html.I(className="fas fa-info-circle ms-2", id="popover-zonas", style={"cursor": "pointer", "color": "#137598"})], className="fw-bold text-center", style={"color": "#026937"})),
            dbc.Popover(dbc.PopoverBody("Para descargar la imagen de la asignación de zonas, da clic derecho sobre ella y selecciona 'Guardar imagen como...'.", style={"fontFamily": "Poppins, sans-serif"}), target="popover-zonas", trigger="hover", placement="top"),
            dbc.CardBody(dbc.Tabs([
                dbc.Tab(
                    label="Cuadro de turnos",
                    tab_id="tab-tabla",
                    label_class_name="fw-bold",
                    children=[dcc.Loading(id="loading-tabla", type="circle", color="#026937", children=html.Div(id="resultados", className="mt-2", style={"minHeight": "148px", "position": "relative"}))],
                    label_style={"color": "#444", "borderBottom": "none", "textDecoration": "none"},
                    active_label_style={"color": "#35944b", "borderBottom": "none", "textDecoration": "none", "boxShadow": "none", "outline": "none"},
                    active_tab_style={"border": "none", "borderBottom": "3px solid #35944b", "backgroundColor": "transparent", "outline": "none", "boxShadow": "none"}
                ),
                dbc.Tab(
                    label="Asignación por zonas",
                    tab_id="tab-grafico",
                    label_class_name="fw-bold",
                    children=[dcc.Loading(id="loading-plot", type="circle", color="#026937", children=html.Div(id="grafico-1", className="mt-2", style={"minHeight": "148px", "position": "relative"}))],
                    label_style={"color": "#444", "borderBottom": "none", "textDecoration": "none"},
                    active_label_style={"color": "#35944b", "borderBottom": "none", "textDecoration": "none", "boxShadow": "none", "outline": "none"},
                    active_tab_style={"border": "none", "borderBottom": "3px solid #35944b", "backgroundColor": "transparent", "outline": "none", "boxShadow": "none"}
                )
            ], id="tabs-resultados", active_tab="tab-tabla"))
        ], className="shadow-sm p-3 rounded")
    ], width=9)
], justify="center", className="mb-4")


# ----------------------------------------
# Callbacks
# ----------------------------------------

# Estilo botones enfoque
@app.callback(
    Output("btn-alta", "style"),
    Output("btn-baja", "style"),
    Input("btn-alta", "n_clicks"),
    Input("btn-baja", "n_clicks")
)
def actualizar_estilos(n1, n2):
    activo = {"backgroundColor": "#35944b", "borderColor": "#35944b", "color": "white", "fontWeight": "bold"}
    inactivo = {"backgroundColor": "white", "borderColor": "#35944b", "color": "#35944b", "fontWeight": "bold"}
    if n1 is None and n2 is None: raise PreventUpdate
    return (activo, inactivo) if n1 >= n2 else (inactivo, activo)

# Alerta por carga .json
@app.callback(
    Output("json-output", "children"),
    Input("upload-json", "contents"),
    Input("upload-json", "filename")
)
def cargar_json(contents, filename):
    if not contents: raise PreventUpdate
    try:
        decoded = base64.b64decode(contents.split(',')[1])
        json.loads(decoded)
        return dbc.Alert(f"Archivo {filename} cargado correctamente", dismissable=True, className="text-center fw-bold", style={"fontSize": "0.75rem", "backgroundColor": "rgba(141, 198, 63, 0.2)", "border": "1px solid #8dc63f", "color": "#4d7a10"})
    except:
        return dbc.Alert("Error al procesar el archivo JSON.", color="danger", dismissable=True, className="text-center")

# Navegación entre páginas
@app.callback(Output('page-content', 'children'), Input('url', 'pathname'))
def navegar(pathname):
    if pathname == '/': return inicio_content
    elif pathname == '/ejecucion': return ejecucion_content
    return html.Div([html.H1("404 - Página no encontrada", className="text-danger"), html.P("La ruta ingresada no existe.")])

# Ejecución
@app.callback(
    Output("resultados", "children"),
    Output("indicador-1", "children"),
    Output("indicador-3", "children"),
    Output("grafico-1", "children"),
    Input("btn-ejecutar", "n_clicks"),
    State("upload-json", "contents"),
    State("upload-json", "filename"),
    State("tiempo-max", "value"),
    State("btn-alta", "n_clicks"),
    State("btn-baja", "n_clicks")
)
def ejecutar(n, contents, filename, tiempo, alta, baja):
    if not n:
        warning = html.Div("Haz clic en 'Ejecutar' para iniciar el proceso", style=dict(color="rgba(249, 161, 44, 0.9)", backgroundColor="rgba(249, 161, 44, 0.15)", textAlign="center", padding="15px", borderRadius="5px", border="2px solid rgba(249, 161, 44, 0.4)", position="absolute", top="50%", left="50%", transform="translate(-50%, -50%)", width="80%", fontWeight="bold"))
        return warning, "-", "-", warning
    if not contents:
        return html.Div("Error: No se ha subido ningún archivo", className="alert alert-danger text-center p-3"), dash.no_update, dash.no_update, dash.no_update
    if not tiempo:
        return html.Div("Establece un tiempo máximo de ejecución en minutos", className="alert alert-info text-center p-3"), dash.no_update, dash.no_update, dash.no_update
    try:
        start_total = time.time()
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        data = json.loads(decoded)
        desks_z = data.get("Desks_Z", {})
        alta_clicks = alta or 0
        baja_clicks = baja or 0
        equit = alta_clicks >= baja_clicks
        start_modelo = time.time()
        obj1, obj2, obj3, meeting, asignaciones, totalFO1, totalFO3 = modelo(data, time_limit=tiempo*60, equit=equit)
        t_modelo = time.time() - start_modelo
        start_tabla = time.time()
        empleados = data.get("Employees_G", [])
        zonas = data.get("Zones", [])
        colores = asignar_colores_zonas(zonas)
        leyenda = construir_leyenda_colores(colores)
        tabla = construir_tabla_html(asignaciones, empleados, colores, meeting)
        grafico = construir_plot_html(asignaciones, empleados, zonas, meeting, desks_z)
        t_tabla = time.time() - start_tabla
        t_total = time.time() - start_total
        tiempo_alert = dbc.Alert([
            html.Div("Ejecución completada", className="mb-1"),
            html.Span(f"Modelo: {t_modelo:.2f}s", className="me-3 fw-bold"),
            html.Span(f"Tabla: {t_tabla:.2f}s", className="me-3 fw-bold"),
            html.Span(f"Total: {t_total:.2f}s", className="fw-bold")
        ], style={"backgroundColor": "rgba(141, 198, 63, 0.15)", "border": "1px solid #8dc63f", "color": "#4d7a10"}, className="text-center mb-3 fw-bold")
        resultados = html.Div([
            tiempo_alert,
            dcc.Markdown(leyenda, dangerously_allow_html=True),
            dcc.Markdown(tabla, dangerously_allow_html=True)
        ], style={'overflowX': 'auto', 'marginTop': '10px'})
        return resultados, grafico_donut(obj1 / totalFO1 * 100, color="#3ebdac"), grafico_donut(obj3 / totalFO3 * 100, color="#0e7774"), grafico
    except Exception as e:
        error_msg = html.Div([
            html.P("Error crítico en la ejecución", className="text-danger fw-bold text-center"),
            html.Pre(f"Tipo de error: {type(e).__name__}\nDetalles: {str(e)}", style={'whiteSpace': 'pre-wrap', 'background': '#f8d7da', 'padding': '15px'})
        ])
        return error_msg, "—", "—", "—"

# ----------------------------------------
# Lanzamiento
# ----------------------------------------
def run_app(): app.run(host="0.0.0.0", port=8050, debug=False)
threading.Thread(target=run_app).start()
print(f"Tu app está corriendo aquí: {ngrok.connect(8050).public_url}")

<IPython.core.display.Javascript object>

Tu app está corriendo aquí: https://9282-34-168-165-240.ngrok-free.app
